In [ ]:
import os
import glob
import pandas as pd

RUTA_LOGS = "."
RUTA_TEST = "."   # donde están los *_test.csv

archivos_xlsx = [
    f for f in glob.glob(os.path.join(RUTA_LOGS, "*.xlsx"))
    if not os.path.basename(f).startswith("~$")
]

archivos_test = glob.glob(os.path.join(RUTA_TEST, "*_test.csv"))

# ---- cargar distribuciones de test ----
dist_test = {}
for ruta in archivos_test:
    nombre = os.path.basename(ruta)
    dataset = nombre.split("_tdataset")[0]
    df_test = pd.read_csv(ruta)
    col_clase = df_test.columns[-1]
    dist_test[dataset] = df_test[col_clase].value_counts()

# ---- análisis principal ----
for ruta in archivos_xlsx:
    nombre_archivo = os.path.basename(ruta)
    dataset = (
        nombre_archivo
        .replace("log_pcsmote_x_muestra_", "")
        .replace(".xlsx", "")
    )

    df = pd.read_excel(ruta, engine="openpyxl")

    columnas = {
        "configuracion",
        "clase_objetivo",
        "synthetics_from_this_seed",
        "es_semilla_valida"
    }
    if not columnas <= set(df.columns):
        raise ValueError(f"Columnas faltantes en {nombre_archivo}")

    print("\n" + "=" * 100)
    print(f"Dataset: {dataset}")
    print("=" * 100)

    for configuracion, bloque in df.groupby("configuracion"):

        print(f"\n{configuracion}")
        print("-" * len(configuracion))

        resumen = (
            bloque
            .groupby("clase_objetivo")
            .agg(
                SG=("synthetics_from_this_seed", "sum"),
                SV=("es_semilla_valida", "sum")
            )
            .sort_index()
        )

        # imprimir por clase
        for clase, fila in resumen.iterrows():
            sg = int(fila["SG"])
            sv = int(fila["SV"])
            ratio = sg / sv if sv > 0 else 0
            print(f"{clase:<6} SV={sv:<3} SG={sg:<4} SG/SV={ratio:.2f}")

        # métricas globales
        total_sg = resumen["SG"].sum()
        cobertura = (resumen["SG"] > 0).sum()
        total_clases = len(resumen)

        print(f"\n  ▶ Total sintéticos (SG): {int(total_sg)}")
        print(f"  ▶ Cobertura de clases  : {cobertura}/{total_clases}")

        if total_sg == 0:
            print("  ⚠ Configuración inválida (SG = 0)")
            continue

        # ---- desalineación con test ----
        if dataset in dist_test:
            print("  ▶ Desalineación SG vs TEST:")
            for clase in resumen.index:
                sg = resumen.loc[clase, "SG"]
                ft = dist_test[dataset].get(clase, 0)
                delta = sg - ft
                print(f"     {clase:<6} SG={sg:<4} TEST={ft:<3} Δ={delta}")


In [ ]:
import os
import re
from pathlib import Path
import matplotlib.pyplot as plt


def _slug(texto: str) -> str:
    """
    Convierte texto a nombre de archivo seguro y estable.
    """
    texto = str(texto).strip().lower()
    texto = re.sub(r"\s+", "_", texto)
    texto = re.sub(r"[^a-z0-9_\-\.]+", "", texto)
    return texto[:180] if len(texto) > 180 else texto


def graficar_sg_sv_por_clase(
    resumen: pd.DataFrame,
    dataset: str,
    configuracion: str,
    directorio_graficos: str = "graficos",
    mostrar_en_notebook: bool = True,
    guardar: bool = True,
    dpi: int = 140,
) -> None:
    """
    Genera:
      1) Burbuja SV vs SG (tamaño ~ SG/SV)
      2) Barra SG/SV por clase

    Guarda en /graficos con nombres determinísticos por dataset+config,
    por lo que se SOBREESCRIBEN al re-ejecutar.

    Espera 'resumen' con índice=clase_objetivo y columnas: SG, SV.
    """

    if resumen is None or len(resumen) == 0:
        print("  ▶ Gráficos: resumen vacío (omitido)")
        return

    if "SG" not in resumen.columns or "SV" not in resumen.columns:
        print("  ▶ Gráficos: faltan columnas SG/SV en resumen (omitido)")
        return

    # -------------------------
    # Preparación de datos
    # -------------------------
    clases = []
    lista_sv = []
    lista_sg = []
    lista_ratio = []

    for clase, fila in resumen.iterrows():
        sg = float(fila["SG"]) if pd.notna(fila["SG"]) else 0.0
        sv = float(fila["SV"]) if pd.notna(fila["SV"]) else 0.0
        ratio = (sg / sv) if sv > 0 else 0.0

        clases.append(str(clase))
        lista_sv.append(sv)
        lista_sg.append(sg)
        lista_ratio.append(ratio)

    # Escala de tamaño (área en puntos^2)
    max_ratio = max(lista_ratio) if len(lista_ratio) > 0 else 0.0
    tamanios = []
    for r in lista_ratio:
        if max_ratio > 0:
            tamanio = 80.0 + (r / max_ratio) * 1320.0
        else:
            tamanio = 80.0
        tamanios.append(tamanio)

    # -------------------------
    # Rutas determinísticas
    # -------------------------
    base = f"{_slug(dataset)}__{_slug(configuracion)}"
    out_dir = Path(directorio_graficos)
    if guardar:
        out_dir.mkdir(parents=True, exist_ok=True)

    ruta_burbuja = out_dir / f"{base}__burbuja_sv_vs_sg.png"
    ruta_barra   = out_dir / f"{base}__barra_sg_sv.png"

    # -------------------------
    # 1) Burbuja (con leyenda)
    # -------------------------
    fig1 = plt.figure()

    # puntos activos (SV>0 y SG>0)
    for i in range(len(clases)):
        if lista_sv[i] > 0 and lista_sg[i] > 0:
            plt.scatter(
                [lista_sv[i]],
                [lista_sg[i]],
                s=[tamanios[i]],
                alpha=0.7,
                label=clases[i]
            )

    # clases sin generación (no se grafican como puntos)
    clases_inactivas = []
    for i in range(len(clases)):
        if not (lista_sv[i] > 0 and lista_sg[i] > 0):
            clases_inactivas.append(clases[i])

    plt.title(f"{dataset} | {configuracion} | Burbuja SV vs SG (tamaño ~ SG/SV)")
    plt.xlabel("SV (semillas válidas)")
    plt.ylabel("SG (sintéticos generados)")
    plt.grid(True, which="both", linestyle="--", linewidth=0.5, alpha=0.5)

    # leyenda fuera del gráfico
    plt.legend(
        loc="upper left",
        bbox_to_anchor=(1.02, 1.0),
        borderaxespad=0.0,
        fontsize=9,
        title="Clase"
    )

    # nota para clases inactivas
    if len(clases_inactivas) > 0:
        texto = "Sin generación (SV=0 o SG=0): " + ", ".join(clases_inactivas)
        plt.figtext(0.01, 0.01, texto, ha="left", va="bottom", fontsize=9)

    # un poco de aire para que no quede pegado al borde
    plt.margins(x=0.12, y=0.15)

    if guardar:
        fig1.savefig(ruta_burbuja, dpi=dpi, bbox_inches="tight")  # sobreescribe

    if mostrar_en_notebook:
        plt.show()

    plt.close(fig1)


    # -------------------------
    # 2) Barra SG/SV
    # -------------------------
    # Orden descendente para lectura
    indices_ordenados = list(range(len(clases)))
    indices_ordenados.sort(key=lambda i: lista_ratio[i], reverse=True)

    clases_ord = []
    ratio_ord = []
    for i in indices_ordenados:
        clases_ord.append(clases[i])
        ratio_ord.append(lista_ratio[i])

    fig2 = plt.figure()
    plt.bar(clases_ord, ratio_ord)

    plt.title(f"{dataset} | {configuracion} | Barra SG/SV por clase")
    plt.xlabel("Clase")
    plt.ylabel("SG/SV")
    plt.xticks(rotation=30, ha="right")
    plt.grid(True, axis="y", linestyle="--", linewidth=0.5, alpha=0.5)
    plt.tight_layout()

    if guardar:
        fig2.savefig(ruta_barra, dpi=dpi, bbox_inches="tight")  # sobreescribe

    if mostrar_en_notebook:
        plt.show()

    plt.close(fig2)


In [11]:
import os
import glob
import pandas as pd

# =========================
# DATASETS A GRAFICAR
# =========================
datasets_a_graficar = {
    "predict_faults",
    # "heart",
    # "wdbc",
}

# =========================
# UTILIDADES
# =========================
def tiene_columnas(df: pd.DataFrame, columnas: list) -> bool:
    for c in columnas:
        if c not in df.columns:
            return False
    return True


def validar_columnas_obligatorias(df: pd.DataFrame, nombre_archivo: str) -> None:
    columnas_obligatorias = [
        "configuracion",
        "clase_objetivo",
        "synthetics_from_this_seed",
        "es_semilla_valida",
    ]

    faltantes = []
    for c in columnas_obligatorias:
        if c not in df.columns:
            faltantes.append(c)

    if len(faltantes) > 0:
        raise ValueError(f"Columnas faltantes en {nombre_archivo}: {faltantes}")


def convertir_a_float_seguro(serie: pd.Series) -> pd.Series:
    """
    Convierte una serie a float de forma robusta:
    - numéricos -> float
    - strings "7/7" -> 1.0
    - strings con coma decimal -> reemplaza coma por punto
    - cualquier otra cosa no convertible -> NaN
    """
    # Paso 1: forzamos a string para procesar patrones, pero preservamos NaN
    s = serie.copy()

    # Normalizamos a string sin perder NaN
    s = s.astype("object")

    valores_convertidos = []

    for v in s.values:
        if pd.isna(v):
            valores_convertidos.append(float("nan"))
            continue

        # Si ya es numérico
        if isinstance(v, (int, float)):
            valores_convertidos.append(float(v))
            continue

        texto = str(v).strip()

        if texto == "":
            valores_convertidos.append(float("nan"))
            continue

        # Caso "a/b"
        if "/" in texto:
            partes = texto.split("/")
            if len(partes) == 2:
                a_txt = partes[0].strip()
                b_txt = partes[1].strip()
                try:
                    a = float(a_txt.replace(",", "."))
                    b = float(b_txt.replace(",", "."))
                    if b != 0:
                        valores_convertidos.append(a / b)
                    else:
                        valores_convertidos.append(float("nan"))
                except Exception:
                    valores_convertidos.append(float("nan"))
                continue

        # Caso decimal con coma
        texto = texto.replace(",", ".")

        try:
            valores_convertidos.append(float(texto))
        except Exception:
            valores_convertidos.append(float("nan"))

    return pd.Series(valores_convertidos, index=serie.index)


# =========================
# CARGA DISTRIBUCIONES TEST
# =========================
def cargar_distribuciones_test(ruta_test: str) -> dict:
    archivos_test = glob.glob(os.path.join(ruta_test, "*_test.csv"))
    dist_test = {}

    for ruta in archivos_test:
        nombre = os.path.basename(ruta)
        dataset = nombre.split("_tdataset")[0]

        df_test = pd.read_csv(ruta)
        col_clase = df_test.columns[-1]
        dist_test[dataset] = df_test[col_clase].value_counts()

    return dist_test


# =========================
# ANÁLISIS 1: SG/SV POR CLASE
# =========================
def construir_resumen_sg_sv_por_clase(bloque: pd.DataFrame) -> pd.DataFrame:
    resumen = (
        bloque
        .groupby("clase_objetivo")
        .agg(
            SG=("synthetics_from_this_seed", "sum"),
            SV=("es_semilla_valida", "sum"),
        )
        .sort_index()
    )
    return resumen


def imprimir_resumen_sg_sv(resumen: pd.DataFrame) -> int:
    for clase, fila in resumen.iterrows():
        sg = int(fila["SG"])
        sv = int(fila["SV"])
        ratio = sg / sv if sv > 0 else 0
        print(f"{clase:<6} SV={sv:<3} SG={sg:<4} SG/SV={ratio:.2f}")

    total_sg = int(resumen["SG"].sum())
    cobertura = int((resumen["SG"] > 0).sum())
    total_clases = int(len(resumen))

    print(f"\n  ▶ Total sintéticos (SG): {total_sg}")
    print(f"  ▶ Cobertura de clases  : {cobertura}/{total_clases}")

    if total_sg == 0:
        print("  ⚠ Configuración inválida (SG = 0)")

    return total_sg


# =========================
# ANÁLISIS 2: DESALINEACIÓN VS TEST
# =========================
def imprimir_desalineacion_vs_test(dataset: str, resumen: pd.DataFrame, dist_test: dict) -> None:
    if dataset not in dist_test:
        return

    print("  ▶ Desalineación SG vs TEST:")
    for clase in resumen.index:
        sg = int(resumen.loc[clase, "SG"])
        ft = int(dist_test[dataset].get(clase, 0))
        delta = sg - ft
        print(f"     {clase:<6} SG={sg:<4} TEST={ft:<3} Δ={delta}")


# =========================
# ANÁLISIS 3: ENTROPÍA (media/std/CV) EN SV=1
# =========================
def analizar_entropia_en_semillas_validas(bloque: pd.DataFrame) -> None:
    # Si el criterio de pureza es "proporcion", entropía NO aplica
    if "criterio_pureza" in bloque.columns:
        criterio = str(bloque["criterio_pureza"].iloc[0]).strip().lower()
        if "prop" in criterio:
            print("  ▶ Entropía: no aplica (criterio_pureza=proporcion)")
            return

    if not tiene_columnas(bloque, ["entropia", "es_semilla_valida"]):
        print("  ▶ Entropía: (omitido, faltan columnas)")
        return

    semillas_validas = bloque[bloque["es_semilla_valida"] == 1]
    if len(semillas_validas) == 0:
        print("  ▶ Entropía: sin semillas válidas")
        return

    ent = convertir_a_float_seguro(semillas_validas["entropia"])
    ent = ent.dropna()

    if len(ent) == 0:
        print("  ▶ Entropía: todos NaN tras conversión")
        return

    media = float(ent.mean())
    std = float(ent.std(ddof=1)) if len(ent) > 1 else 0.0
    cv = (std / media) if media != 0 else 0.0

    print("  ▶ Entropía (solo SV=1):")
    print(f"     n={len(ent)} | media={media:.4f} | std={std:.4f} | CV={cv:.4f}")


# =========================
# ANÁLISIS 4: CORRELACIÓN entropía ↔ SG_por_semilla (Spearman) EN SV=1
# =========================
def analizar_correlacion_entropia_vs_sg(bloque: pd.DataFrame) -> None:

    # Si el criterio de pureza es "proporcion", entropía NO aplica
    if "criterio_pureza" in bloque.columns:
        criterio = str(bloque["criterio_pureza"].iloc[0]).strip().lower()
        if "prop" in criterio:
            print("  ▶ Correlación entropía↔SG: no aplica (criterio_pureza=proporcion)")
            return

    cols = ["entropia", "synthetics_from_this_seed", "es_semilla_valida"]
    if not tiene_columnas(bloque, cols):
        print("  ▶ Correlación entropía↔SG: (omitido, faltan columnas)")
        return

    semillas_validas = bloque[bloque["es_semilla_valida"] == 1].copy()
    if len(semillas_validas) < 3:
        print("  ▶ Correlación entropía↔SG: datos insuficientes (n<3)")
        return

    x = convertir_a_float_seguro(semillas_validas["entropia"])
    y = convertir_a_float_seguro(semillas_validas["synthetics_from_this_seed"])

    # Alineamos y tiramos NaN
    tmp = pd.DataFrame({"x": x, "y": y}).dropna()
    if len(tmp) < 3:
        print("  ▶ Correlación entropía↔SG: datos insuficientes tras limpieza")
        return

    corr = tmp["x"].corr(tmp["y"], method="spearman")
    if pd.isna(corr):
        print("  ▶ Correlación entropía↔SG: no calculable")
        return

    print("  ▶ Correlación (Spearman) entropía ↔ SG_por_semilla (solo SV=1):")
    print(f"     rho={float(corr):.4f}")


# =========================
# ANÁLISIS 5: EFICIENCIA DE SEMILLAS (SV activas)
# =========================
def analizar_eficiencia_de_semillas(bloque: pd.DataFrame) -> None:
    cols = ["es_semilla_valida", "synthetics_from_this_seed"]
    if not tiene_columnas(bloque, cols):
        print("  ▶ Eficiencia semillas: (omitido, faltan columnas)")
        return

    semillas_validas = bloque[bloque["es_semilla_valida"] == 1].copy()
    if len(semillas_validas) == 0:
        print("  ▶ Eficiencia semillas: sin semillas válidas")
        return

    total_sv = int(len(semillas_validas))

    sg_por_semilla = convertir_a_float_seguro(semillas_validas["synthetics_from_this_seed"])
    semillas_validas = semillas_validas.copy()
    semillas_validas["sg_float"] = sg_por_semilla

    activas = semillas_validas[semillas_validas["sg_float"] > 0]
    cant_activas = int(len(activas))
    pct_activas = (cant_activas / total_sv) if total_sv > 0 else 0.0

    if cant_activas > 0:
        media_sg_activas = float(activas["sg_float"].mean())
        mediana_sg_activas = float(activas["sg_float"].median())
    else:
        media_sg_activas = 0.0
        mediana_sg_activas = 0.0

    print("  ▶ Eficiencia de semillas (solo SV=1):")
    print(f"     SV_totales={total_sv} | SV_activas={cant_activas} ({pct_activas*100:.2f}%)")
    print(f"     SG_por_semilla_activa: media={media_sg_activas:.3f} | mediana={mediana_sg_activas:.3f}")


# =========================
# ANÁLISIS 6: SV=1 vs SV=0 (entropia/densidad/riesgo)
# =========================
def analizar_diferencias_sv_vs_no_sv(bloque: pd.DataFrame) -> None:
    if "es_semilla_valida" not in bloque.columns:
        print("  ▶ Comparación SV vs no-SV: (omitido, falta es_semilla_valida)")
        return

    columnas = []
    for c in ["entropia", "densidad", "riesgo"]:
        if c in bloque.columns:
            columnas.append(c)

    if len(columnas) == 0:
        print("  ▶ Comparación SV vs no-SV: (omitido, faltan entropia/densidad/riesgo)")
        return

    sv = bloque[bloque["es_semilla_valida"] == 1]
    no_sv = bloque[bloque["es_semilla_valida"] == 0]

    if len(sv) == 0 or len(no_sv) == 0:
        print("  ▶ Comparación SV vs no-SV: datos insuficientes (uno de los grupos vacío)")
        return

    print("  ▶ Diferencias SV=1 vs SV=0 (media y std):")
    for c in columnas:
        v1 = convertir_a_float_seguro(sv[c]).dropna()
        v0 = convertir_a_float_seguro(no_sv[c]).dropna()

        if len(v1) == 0 or len(v0) == 0:
            print(f"     {c:<8} (omitido: NaN tras conversión)")
            continue

        media_1 = float(v1.mean())
        std_1 = float(v1.std(ddof=1)) if len(v1) > 1 else 0.0

        media_0 = float(v0.mean())
        std_0 = float(v0.std(ddof=1)) if len(v0) > 1 else 0.0

        print(f"     {c:<8} SV=1 media={media_1:.4f} std={std_1:.4f} | SV=0 media={media_0:.4f} std={std_0:.4f}")


# =========================
# ANÁLISIS 7: FRONTERA “REAL” vs SEMILLAS MUERTAS (por criterio interno)
# =========================
def analizar_frontera_por_criterio(bloque: pd.DataFrame) -> None:
    # -------------------------
    # 1) Validación base
    # -------------------------
    columnas_base = [
        "pasa_pureza", "pasa_densidad", "pasa_riesgo",
        "densidad", "riesgo"
    ]

    for col in columnas_base:
        if col not in bloque.columns:
            print("  ▶ Frontera por criterio: (omitido, faltan columnas)")
            return

    # -------------------------
    # 2) Elegir variable para pureza según criterio_pureza
    # -------------------------
    col_valor_pureza = "entropia"  # default

    if "criterio_pureza" in bloque.columns:
        criterio = str(bloque["criterio_pureza"].iloc[0]).strip().lower()

        # Ajustá esta condición si tu campo tiene otros nombres
        if "prop" in criterio:
            col_valor_pureza = "proporcion_min_valor"
        else:
            col_valor_pureza = "entropia"

    # Validación específica de la columna de pureza elegida
    if col_valor_pureza not in bloque.columns:
        print(f"  ▶ Frontera por criterio: (omitido, falta {col_valor_pureza})")
        return

    # -------------------------
    # 3) Definir criterios a analizar
    # -------------------------
    criterios = [
        {"nombre": "pureza",   "col_pasa": "pasa_pureza",   "col_valor": col_valor_pureza},
        {"nombre": "densidad", "col_pasa": "pasa_densidad", "col_valor": "densidad"},
        {"nombre": "riesgo",   "col_pasa": "pasa_riesgo",   "col_valor": "riesgo"},
    ]

    print("  ▶ Análisis de frontera por criterio:")

    # -------------------------
    # 4) Ejecutar análisis: pasa vs no_pasa
    # -------------------------
    for crit in criterios:
        nombre = crit["nombre"]
        col_pasa = crit["col_pasa"]
        col_valor = crit["col_valor"]

        pasa = bloque[bloque[col_pasa] == 1][col_valor]
        no_pasa = bloque[bloque[col_pasa] == 0][col_valor]

        pasa_num = convertir_a_float_seguro(pasa).dropna()
        no_num = convertir_a_float_seguro(no_pasa).dropna()

        if len(pasa_num) == 0 or len(no_num) == 0:
            print(f"     {nombre:<8}: datos insuficientes (NaN tras conversión)")
            continue

        media_pasa = float(pasa_num.mean())
        std_pasa = float(pasa_num.std(ddof=1)) if len(pasa_num) > 1 else 0.0

        media_no = float(no_num.mean())
        std_no = float(no_num.std(ddof=1)) if len(no_num) > 1 else 0.0

        delta_media = media_pasa - media_no

        etiqueta_valor = col_valor
        print(
            f"     {nombre:<8} [{etiqueta_valor}] | "
            f"pasa: n={len(pasa_num):<4} μ={media_pasa:.4f} σ={std_pasa:.4f} | "
            f"no pasa: n={len(no_num):<4} μ={media_no:.4f} σ={std_no:.4f} | "
            f"Δμ={delta_media:.4f}"
        )


# =========================
# EJECUCIÓN PRINCIPAL
# =========================
RUTA_LOGS = "."
RUTA_TEST = "."

archivos_xlsx = [
    f for f in glob.glob(os.path.join(RUTA_LOGS, "*.xlsx"))
    if not os.path.basename(f).startswith("~$")
]

dist_test = cargar_distribuciones_test(RUTA_TEST)

for ruta in archivos_xlsx:
    nombre_archivo = os.path.basename(ruta)
    dataset = (
        nombre_archivo
        .replace("log_pcsmote_x_muestra_", "")
        .replace(".xlsx", "")
    )

    df = pd.read_excel(ruta, engine="openpyxl")
    validar_columnas_obligatorias(df, nombre_archivo)

    if dataset in datasets_a_graficar:
        print("\n" + "=" * 100)
        print(f"Dataset: {dataset}")
        print("=" * 100)

    for configuracion, bloque in df.groupby("configuracion"):

        if dataset not in datasets_a_graficar:
            continue
        
        print(f"\n{configuracion}")
        print("-" * len(configuracion))

        # 1) Resumen SG/SV por clase
        resumen = construir_resumen_sg_sv_por_clase(bloque)
        total_sg = imprimir_resumen_sg_sv(resumen)

        # 2) Desalineación vs TEST
        if total_sg > 0:
            imprimir_desalineacion_vs_test(dataset, resumen, dist_test)
            titulo = f"{dataset} | {configuracion}"
            
            graficar_sg_sv_por_clase(
                resumen=resumen,
                dataset=dataset,
                configuracion=configuracion,
                directorio_graficos="graficos",
                mostrar_en_notebook=False,
                guardar=True
            )

        # 3) Entropía (dispersión) en semillas válidas
        analizar_entropia_en_semillas_validas(bloque)

        # 4) Correlación entropía vs SG_por_semilla
        analizar_correlacion_entropia_vs_sg(bloque)

        # 5) Eficiencia de semillas
        analizar_eficiencia_de_semillas(bloque)

        # 6) SV=1 vs SV=0
        analizar_diferencias_sv_vs_no_sv(bloque)

        # 7) Frontera por criterio (pasa_* vs no pasa_*)
        analizar_frontera_por_criterio(bloque)



Dataset: predict_faults

PRD85_PR40_CPent_UD050_UR045_PE60_I0
------------------------------------
Heat Dissipation Failure SV=12  SG=7632 SG/SV=636.00
Overstrain Failure SV=4   SG=7660 SG/SV=1915.00
Power Failure SV=14  SG=7646 SG/SV=546.14
Random Failures SV=0   SG=0    SG/SV=0.00
Tool Wear Failure SV=0   SG=0    SG/SV=0.00

  ▶ Total sintéticos (SG): 22938
  ▶ Cobertura de clases  : 3/5
  ▶ Entropía (solo SV=1):
     n=30 | media=0.6863 | std=0.2648 | CV=0.3859
  ▶ Correlación (Spearman) entropía ↔ SG_por_semilla (solo SV=1):
     rho=0.6154
  ▶ Eficiencia de semillas (solo SV=1):
     SV_totales=30 | SV_activas=30 (100.00%)
     SG_por_semilla_activa: media=764.600 | mediana=610.000
  ▶ Diferencias SV=1 vs SV=0 (media y std):
     entropia SV=1 media=0.6863 std=0.2648 | SV=0 media=0.5984 std=0.4030
     densidad SV=1 media=0.9429 std=0.1100 | SV=0 media=0.8381 std=0.3021
     riesgo   SV=1 media=0.0810 std=0.0970 | SV=0 media=0.2903 std=0.3025
  ▶ Análisis de frontera por criterio